In [1]:
pip install face-recognition


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import face_recognition
import cv2
import numpy as np
import csv
import os
from datetime import datetime
import time

# -------------------- Configuration --------------------
PHOTO_DIR = "photos"  # Folder containing known face images
AUTO_EXIT_ON_COMPLETION = True  # Set to False to disable auto-exit
# -------------------------------------------------------

# Initialize webcam
video_capture = cv2.VideoCapture(0)
time.sleep(2)  # Give time for the camera to warm up

# Load known face encodings and names
known_face_encodings = []
known_face_names = []

for filename in os.listdir(PHOTO_DIR):
    if filename.endswith(('.jpg', '.png', '.jpeg')):
        name = os.path.splitext(filename)[0]  # Remove file extension
        image_path = os.path.join(PHOTO_DIR, filename)
        image = face_recognition.load_image_file(image_path)
        encodings = face_recognition.face_encodings(image)
        if encodings:
            known_face_encodings.append(encodings[0])
            known_face_names.append(name)
        else:
            print(f"[Warning] No face found in image: {filename}")

# Track attendance
students_remaining = set(known_face_names)
attendance_log = []

# Create CSV with current date
now = datetime.now()
current_date = now.strftime("%Y-%m-%d")
csv_filename = current_date + '.csv'

with open(csv_filename, 'w', newline='') as f:
    csv_writer = csv.writer(f)
    csv_writer.writerow(['Name', 'Time'])  # CSV header

    while True:
        ret, frame = video_capture.read()
        if not ret:
            print("[Error] Failed to capture frame from webcam.")
            break

        # Resize frame for faster processing
        small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)
        rgb_small_frame = small_frame[:, :, ::-1]

        # Detect faces and recognize
        face_locations = face_recognition.face_locations(rgb_small_frame)
        face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)
        face_names = []

        for face_encoding in face_encodings:
            matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
            name = "Unknown"

            face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
            best_match_index = np.argmin(face_distances)

            if matches[best_match_index]:
                name = known_face_names[best_match_index]

            face_names.append(name)

            if name != "Unknown" and name in students_remaining:
                students_remaining.remove(name)
                current_time = datetime.now().strftime("%H:%M:%S")
                csv_writer.writerow([name, current_time])
                f.flush()
                print(f"[Info] Marked present: {name} at {current_time}")

        # Display annotations
        for (top, right, bottom, left), name in zip(face_locations, face_names):
            # Scale back up face locations
            top *= 4
            right *= 4
            bottom *= 4
            left *= 4

            # Draw box
            color = (0, 255, 0) if name != "Unknown" else (0, 0, 255)
            cv2.rectangle(frame, (left, top), (right, bottom), color, 2)

            # Draw label
            cv2.rectangle(frame, (left, bottom - 35), (right, bottom), color, cv2.FILLED)
            font = cv2.FONT_HERSHEY_DUPLEX
            cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)

        # Show frame
        cv2.imshow("Face Recognition Attendance", frame)

        # Break loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            print("[Exit] 'q' key pressed.")
            break

        # Auto-exit if all students marked present
        if AUTO_EXIT_ON_COMPLETION and not students_remaining:
            print("[Info] All students have been marked present.")
            break

# Clean up
video_capture.release()
cv2.destroyAllWindows()
print(f"[Saved] Attendance recorded in '{csv_filename}'")


[Info] Marked present: dhoni at 00:19:30
[Info] Marked present: tata at 00:20:03
[Exit] 'q' key pressed.
[Saved] Attendance recorded in '2025-06-17.csv'
